In [1]:
import pandas as pd
import haversine
import json
import numpy as np

In [2]:
zip_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/sales_by_zip_by_store_0330.csv",dtype=str)
zip_by_store=zip_by_store[['location_id','zip','revenue_flag']]

In [3]:
df_SOTF=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0801_SOTF/Aug_SOTF.xlsx",dtype=str)

In [4]:
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_2=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list_1['store_zip']=store_list_1['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_2['store_zip']=store_list_2['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

store_list_1=store_list_1[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_2=store_list_2[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_1=store_list_1[~store_list_1['location_id'].isin(store_list_2['location_id'])]
store_list=store_list_2.append(store_list_1)
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]


store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [5]:
df_old_stores=store_list[store_list['location_id'].isin(df_SOTF['location_id'])]


In [6]:
def merge_zip_selection(df):
    df=pd.merge(df,zip_by_store,on="location_id",how="left")
    df=df[df['revenue_flag'].isin(["P","S"])]
    df['distance']=np.nan
    df=df.reset_index()
    del df['index']
    for i in range(len(df)):
        store_loc=(df['latitude_meas'][i],df['longitude_meas'][i])
        try:
            df['distance'][i]=round(haversine.haversine(store_loc,zip_centers[df['zip'][i]],miles=True),4)
        except:
            df['distance'][i]="NA"
            print(df['zip'][i])
            
        
    return df

In [7]:
old_store_output=merge_zip_selection(df_old_stores)
old_store_output=old_store_output.sort_values(['location_id','revenue_flag'])
del old_store_output['longitude_meas']
del old_store_output['latitude_meas']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


77614
95937
05657


In [8]:
new_stores=[x for x in df_SOTF['location_id'].tolist() if x not in store_list['location_id'].tolist()]

In [9]:
new_stores_df=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0801_SOTF/Aug_SOTF_New_Stores.xlsx",dtype=str)
new_stores_df=new_stores_df[['New or Relo Locations','Zip']].rename(columns={"New or Relo Locations":"location_id"})
new_stores_df['Zip']=new_stores_df['Zip'].apply(lambda x: x.split("-")[0].zfill(5))

In [10]:
new_stores

['5367', '5365', '5369', '4680', '5377']

In [11]:
new_store_output=pd.DataFrame()
for store in new_stores:
    store_zip=new_stores_df[new_stores_df['location_id']==store]['Zip'].unique().tolist()[0]
    for zip_cd in list(zip_centers.keys()):
        dist=haversine.haversine(zip_centers[zip_cd],zip_centers[store_zip],miles=True)
        i=0
        if dist<=10:
            df_app=pd.DataFrame({"location_id":store,"store_zip":store_zip,'zip':zip_cd,'revenue_flag':'new_in_10_miles','distance':dist},index=[i])
            i=i+1
            new_store_output=new_store_output.append(df_app)
new_store_output=new_store_output[old_store_output.columns.tolist()]
output=old_store_output.append(new_store_output)

In [12]:
new_store_not_in_TA_with_latlong=[x for x in df_SOTF['location_id'].tolist() if x not in output['location_id'].tolist()]

In [13]:
add_new_store_output=pd.DataFrame()

for store in new_store_not_in_TA_with_latlong:
    stoer_lat=store_list[store_list['location_id']==store]['latitude_meas'].unique()[0]
    stoer_long=store_list[store_list['location_id']==store]['longitude_meas'].unique()[0]
    store_zip=store_list[store_list['location_id']==store]['store_zip'].unique()[0]
    store_center=(stoer_lat,stoer_long)
    for zip_cd in list(zip_centers.keys()):
        dist=haversine.haversine(zip_centers[zip_cd],store_center,miles=True)
        i=0
        if dist<=10:
            df_app=pd.DataFrame({"location_id":store,"store_zip":store_zip,'zip':zip_cd,'revenue_flag':'new_in_10_miles','distance':dist},index=[i])
            i=i+1
            add_new_store_output=add_new_store_output.append(df_app)
add_new_store_output=add_new_store_output[old_store_output.columns.tolist()]
output=output.append(add_new_store_output)    

In [14]:
unique_P_list=output[output['revenue_flag']=="P"]['zip'].unique().tolist()
unique_S_list=output[output['revenue_flag']=="S"]['zip'].unique().tolist()
unique_S_list=[x for x in unique_S_list if x not in unique_P_list]
unique_new_list=output[output['revenue_flag']=="new_in_10_miles"]['zip'].unique().tolist()
unique_new_list=[x for x in unique_new_list if x not in unique_P_list]
unique_new_list=[x for x in unique_new_list if x not in unique_S_list]
row_num=max(len(unique_P_list),len(unique_S_list),len(unique_new_list))

unique_zips_output=pd.DataFrame({"P_Zips":unique_P_list+[np.nan]*(row_num-len(unique_P_list)),
                                "S_Zips":unique_S_list+[np.nan]*(row_num-len(unique_S_list)),
                                "new_Zips":unique_new_list+[np.nan]*(row_num-len(unique_new_list))},
                               index=range(row_num))

In [16]:
store_zip=output[['location_id','store_zip']].drop_duplicates()

In [18]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0801_SOTF/Zips_for_SOTF_Aug_JL_20180802_53_Stores.xlsx",engine="xlsxwriter")
unique_zips_output.to_excel(writer,"unique_zips",index=False)
output.to_excel(writer,"zips_by_Store",index=False)
store_zip.to_excel(writer,"store_zip",index=False)
writer.save()